In [48]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import warnings

warnings.filterwarnings("ignore")
pio.templates.default = "plotly_white"

In [49]:
data = pd.read_csv(
    "/home/pydev/workflow/dt_learn_data_science/public_projects/recesion_analysis/UK_monthly_gdp.csv"
)
data.head()


,Time Period,GDP Growth
0,/01/2020,0.3
1,/02/2020,-0.5
2,/03/2020,-7.0
3,/04/2020,-20.9
4,/05/2020,3.2


In [50]:
fig = go.Figure(
    data=go.Heatmap(
        z=[data["GDP Growth"]], x=data.index, y=["GDP Growth"], colorscale="Viridis"
    )
)

fig.update_layout(
    title="GDP Growth Over Time", xaxis_title="Time Period", yaxis_title=""
)
fig.show()

In [51]:
# Convert monthly data to quarterly data using resample method
data["Time Period"] = pd.to_datetime(data["Time Period"], format="/%m/%Y")
data.set_index("Time Period", inplace=True)
quarterly_data = data.resample("Q").mean()
quarterly_data.head(14)

,GDP Growth
Time Period,
2020-03-31,-2.400000
2020-06-30,-2.900000
2020-09-30,3.500000
2020-12-31,0.200000
2021-03-31,0.033333
2021-06-30,1.800000
2021-09-30,0.433333
2021-12-31,0.400000
2022-03-31,0.066667


In [52]:
# Calculate recession based on quarterly GDP growth
quarterly_data["Recession"] = (quarterly_data["GDP Growth"] < 0) & (
    quarterly_data["GDP Growth"].shift(1) < 0
)
# Fill missing values with False (since the first quarter cannot be in a recession)
quarterly_data["Recession"].fillna(False, inplace=True)
# quarterly_data.head(14)
# Plot the GDP growth and recession data
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=quarterly_data.index,
        y=quarterly_data["GDP Growth"],
        name="GDP Growth",
        line=dict(color="green", width=2),
    )
)
fig.add_trace(
    go.Scatter(
        x=quarterly_data[quarterly_data["Recession"]].index,
        y=quarterly_data[quarterly_data["Recession"]]["GDP Growth"],
        name="Recession",
        line=dict(color="red", width=2),
    )
)

fig.update_layout(
    title="GDP Growth and Recession over Time (Quarterly Data)",
    xaxis_title="Time Period",
    yaxis_title="GDP Growth",
)

fig.show()

In [55]:
# The red line shows the periods of negative GDP growth (considered recessions), and the green line 
# shows the overall trend in GDP growth over time.

# Let us now analyze the severity of the recession. The severity of a recession refers to the extent to 
# which the economy contracts during a recession. A severe recession involves a deeper and more prolonged
# decline in economic activity, resulting in negative effects on employment, incomes and other economic 
# indicators. Here’s how to analyze the severity of the recession:

quarterly_data["Recession Start"] = quarterly_data["Recession"].ne(quarterly_data["Recession"].shift()).cumsum()

quarterly_data.head(14)

,GDP Growth,Recession,Recession Start
Time Period,,,
2020-03-31,-2.400000,False,1
2020-06-30,-2.900000,True,2
2020-09-30,3.500000,False,3
2020-12-31,0.200000,False,3
2021-03-31,0.033333,False,3
2021-06-30,1.800000,False,3
2021-09-30,0.433333,False,3
2021-12-31,0.400000,False,3
2022-03-31,0.066667,False,3
